In [31]:
import sys
sys.path.append("..")

from pytorch.model import LModel
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn import metrics
import torch
from torch.autograd import Variable
import numpy as np

In [77]:
ANNOTATED_TEST_DATASET = '../data/mozgalo_split/validation'
MODEL_PATH = "../models/epoch_07-valLoss_0.00454-valAcc_0.99937"
CPU_CORES = 2
BATCH_SIZE = 2
NUM_CLASSES = 26
USE_GPU = False

In [90]:
def normalize(image):
    image /= 255.
    image -= 0.5

    return image


def crop_upper_part(image, percent=0.4):
    height, _, _ = image.shape
    point = int(percent * height)
    return image[:point]

def data_transformations(input_shape, crop_perc = 0.5):
    return transforms.Compose([
        transforms.Lambda(lambda x: crop_upper_part(np.array(x), crop_perc)),
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.Resize((input_shape[1], input_shape[2])),
        transforms.Lambda(lambda x: normalize(np.array(x, dtype=np.float32))),
        transforms.ToTensor(),
    ])

def var(tensor, volatile=False):
    if USE_GPU:
        tensor = tensor.cuda(0)
    return Variable(tensor, volatile=volatile)

def test(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        y_pred = logit.max(1)[1]
        
        predictions.extend([x.data[0] for x in y_pred])
        ground_truths.extend([x.data[0] for x in test_y])
        
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
        if i == 1:
            break
        
    return predictions, ground_truths
    

In [89]:
# Load model
model = LModel(margin=1, num_classes=NUM_CLASSES, fine_tune=True)


model.load_state_dict(torch.load(MODEL_PATH, map_location=lambda storage, loc: storage))
print("Loaded model from:", MODEL_PATH)


if USE_GPU:
    model.cuda(0)

Loaded model from: ../models/epoch_07-valLoss_0.00454-valAcc_0.99937


/home/bartol/.virtual_envs/mozgalo/lib/python3.6/site-packages/torch/nn/modules/module.py:514: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  own_state[name].copy_(param)


In [79]:
test_transform = data_transformations(model.model.input_size)
test_dataset = datasets.ImageFolder(root=ANNOTATED_TEST_DATASET, transform=test_transform)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    num_workers=CPU_CORES)

In [91]:
pred, true = test(model, test_dataset_loader)

In [92]:
print(pred, true)

[24, 24, 24, 24] [24, 21, 24, 3]


In [62]:
report = metrics.classification_report(true, pred)
print(report)

             precision    recall  f1-score   support

         18       0.00      0.00      0.00         1
         24       0.00      0.00      0.00         0
         25       0.00      0.00      0.00         5

avg / total       0.00      0.00      0.00         6



/home/bartol/.virtual_envs/mozgalo/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bartol/.virtual_envs/mozgalo/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
